In [ ]:
import pandas as pd
import numpy as np

def preprocess_ais_data(df, mode="strict"):
    # Ubah kolom waktu menjadi datetime
    df['dt_pos_utc'] = pd.to_datetime(df['dt_pos_utc'], errors='coerce')
    df['dt_insert_utc'] = pd.to_datetime(df['dt_insert_utc'], errors='coerce')

    # Step 1 – Validasi skema: buang baris dengan nilai kosong di kolom kunci dan pastikan berada dalam rentang yang sah
    required_cols = ['mmsi', 'dt_pos_utc', 'latitude', 'longitude', 'sog']
    df = df.dropna(subset=required_cols)
    df = df[df['mmsi'].between(100_000_000, 999_999_999)]
    df = df[df['latitude'].between(-90, 90)]
    df = df[df['longitude'].between(-180, 180)]

    # Imputasi nilai IMO yang kosong berdasarkan MMSI
    if 'imo' in df.columns:
        df['imo'] = df.groupby('mmsi')['imo'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))

    # Step 2 – Deduplikasi: buang data yang memiliki Primary Key ganda
    df = df.drop_duplicates(subset=['mmsi', 'dt_pos_utc', 'message_type'])

    # Step 3 – Filter domain: filter nilai draught dan jenis kapal, atau beri flag jika mode "relaxed"
    if mode == "strict":
        if 'draught' in df.columns:
            df = df[df['draught'].notna() & (df['draught'] > 0)]
        if 'vessel_type' in df.columns:
            df = df[df['vessel_type'].isin(['Cargo', 'Tanker'])]
    else:
        # Jika relaxed, tandai saja baris yang bermasalah tanpa menghapusnya
        df['draught_flag'] = (df['draught'] == 0) | (df['draught'].isna())
        df['vessel_type_flag'] = ~df['vessel_type'].isin(['Cargo', 'Tanker']) if 'vessel_type' in df.columns else False

    # Step 4 – Validasi nilai sog dan timestamp
    df = df[df['sog'].between(0, 70)]
    if df['dt_pos_utc'].dt.tz is None:
        df['dt_pos_utc'] = df['dt_pos_utc'].dt.tz_localize('UTC')
    df = df[df['dt_pos_utc'] <= pd.Timestamp.now(tz='UTC') + pd.Timedelta(minutes=5)]

    # Step 5 – Audit metadata: temukan kapal dengan metadata inkonsisten (IMO, nama, panjang, lebar)
    meta_flags = df.groupby('mmsi')[['imo', 'vessel_name', 'length', 'width']].nunique()
    meta_inconsistent = meta_flags[(meta_flags > 1).any(axis=1)]

    # Step 6 – Deteksi loncatan spasial dan jeda sinyal antar titik
    df = df.sort_values(by=['mmsi', 'dt_pos_utc'])
    df['prev_lat'] = df.groupby('mmsi')['latitude'].shift()
    df['prev_lon'] = df.groupby('mmsi')['longitude'].shift()
    df['prev_time'] = df.groupby('mmsi')['dt_pos_utc'].shift()

    # Fungsi untuk menghitung jarak antara dua koordinat menggunakan rumus Haversine
    def haversine(lat1, lon1, lat2, lon2):
        r = 6371  # radius bumi dalam km
        lat1, lat2, lon1, lon2 = map(np.radians, [lat1, lat2, lon1, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
        return 2 * r * np.arcsin(np.sqrt(a))

    # Hitung jarak antar titik dan kecepatan perpindahan
    df['jump_km'] = haversine(df['prev_lat'], df['prev_lon'], df['latitude'], df['longitude'])
    df['dt_diff_min'] = (df['dt_pos_utc'] - df['prev_time']).dt.total_seconds() / 60
    df['km_per_min'] = df['jump_km'] / df['dt_diff_min']

    # Hapus data teleport jika mode strict, atau tandai jika mode relaxed
    if mode == "strict":
        df = df[~(df['km_per_min'] > 30)]
    else:
        df['jump_flag'] = df['km_per_min'] > 30

    # Tandai sinyal yang hilang lebih dari 60 menit
    df['gap_flag'] = df['dt_diff_min'] > 60

    # Step 7 – Tag inside_port: dummy kondisi jika draught > 5 (placeholder untuk spatial join)
    df['inside_port'] = df['draught'] > 5 if 'draught' in df.columns else False

    return df.reset_index(drop=True), meta_inconsistent


In [ ]:
def quality_metrics(df):
    results = {}
    N = len(df)  # total jumlah baris

    # Indikator 1 – Persentase nilai kosong (missing values)
    results['missing_pct_mmsi'] = df['mmsi'].isna().mean() * 100
    results['missing_pct_dt'] = df['dt_pos_utc'].isna().mean() * 100
    results['missing_pct_coord'] = df[['latitude', 'longitude']].isna().mean().mean() * 100
    results['missing_pct_sog'] = df['sog'].isna().mean() * 100

    # Indikator 2 – Persentase MMSI tidak valid
    results['invalid_mmsi_pct'] = (~df['mmsi'].between(100_000_000, 999_999_999)).mean() * 100

    # Indikator 3 – Draught = 0
    if 'draught' in df.columns:
        results['zero_draught_pct'] = (df['draught'] == 0).mean() * 100

    # Indikator 4 – Jumlah rekor dan jumlah kapal unik
    results['n_records'] = N
    results['n_unique_mmsi'] = df['mmsi'].nunique()

    # Indikator 5 – Rata-rata ping per kapal
    results['avg_rows_per_mmsi'] = results['n_records'] / max(1, results['n_unique_mmsi'])

    # Indikator 6 – Distribusi ping per hari dalam seminggu
    if 'dt_pos_utc' in df.columns:
        df['weekday'] = df['dt_pos_utc'].dt.day_name()
        weekday_dist = df['weekday'].value_counts().to_dict()
        results.update({f"weekday_{k}": v for k, v in weekday_dist.items()})

    # Indikator 7 – Persentase data yang lolos dari validasi rentang
    range_mask = df['latitude'].between(-90, 90) & df['longitude'].between(-180, 180) & df['sog'].between(0, 70)
    results['schema_range_pass_rate'] = range_mask.mean() * 100

    return pd.DataFrame([results])
